In [ ]:
import segmentation_models_pytorch as smp
import torch
import os
import cv2
import timm
import albumentations as A
from skimage import io
import numpy as np
import segmentation_models_pytorch as smp
from skimage import io

In [ ]:
timm.list_models(pretrained=True)

In [ ]:
transform = A.Compose([
    A.Resize(width=256, height=256)
])

In [ ]:
model = smp.Unet(encoder_name = "timm-resnest50d", classes = 2, in_channels = 3)
model.eval()

In [ ]:
model.load_state_dict(torch.load(r"E:\source\repos\VisualDL\timm-resnest50d, Unet.pt"))

In [ ]:
image_dir = r"C:\Users\phili\Downloads\Telegram Desktop\Cells\Cells\test\images"

In [ ]:
for cnt, im in enumerate(os.listdir(image_dir)):
    im = io.imread(os.path.join(image_dir, im))
    im = im/255.
    im = transform(image = im)["image"]
    im = torch.tensor(im, dtype = torch.float).permute(2, 0, 1).unsqueeze(0)
    print(im.shape)
    pred = model(im)
    pred = torch.argmax(pred, 1)[0] * 255.
    cv2.imwrite(str(cnt) + ".png", pred.numpy())

In [ ]:
ii = cv2.imread(r"C:\Users\phili\Downloads\Telegram Desktop\Cells1\Cells\train\labels\05__1_4685_10225.png",0)

In [ ]:
ii = ii * 255.

In [ ]:
cv2.imwrite("test.png", ii)

# Replace 2D with 3D

In [ ]:
from torch import nn

In [ ]:
for name, modules in model.named_modules():
    for module in modules:
        print(module)
    if(isinstance(module, nn.Conv2d)):
        kernel_size = module.kernel_size[0]
        stride = module.stride[0]
        padding = module.padding[0]
        weight = module.weight.unsqueeze(2) / kernel_size
        weight = torch.cat([weight for _ in range(0, kernel_size)], dim=2)
        bias = module.bias

        if(bias is None):
            print(modules)
            print(modules[name])
            modules[name] = nn.Conv3d(in_channels=module.weight.shape[1], out_channels=module.weight.shape[0],
                               kernel_size=kernel_size, padding=padding, stride=stride, bias=False)
        else:
            modules[name] = nn.Conv3d(in_channels=module.weight.shape[1], out_channels=module.weight.shape[0],
                               kernel_size=kernel_size, padding=padding, stride=stride, bias=True)
            modules[name].bias = bias

            modules[name].weight.data = weight

    elif(isinstance(module, nn.BatchNorm2d)):
        weight = module.weight
        bias = module.bias
        modules[name] = nn.BatchNorm3d(weight.shape[0])
        modules[name].weight = weight
        modules[name].bias = bias

for name in modules:
    parent_module = model
    objs = name.split(".")
    if len(objs) == 1:
        model.__setattr__(name, modules[name])
        continue

    for obj in objs[:-1]:
        parent_module = parent_module.__getattr__(obj)

    parent_module.__setattr__(objs[-1], modules[name])

In [ ]:
def get_all_modules(module):
    for a in module:
        get_all_modules(a.children())
        if isinstance(a, nn.Conv2d):
            print(a)
            #a = nn.Conv3d(3,32,3)
            
        elif isinstance(a, nn.BatchNorm2d):
            print(a)

            
  



In [ ]:
get_all_modules(model.children())

In [ ]:
for a in model.encoder.named_modules():
    print(a)
    print()

In [ ]:
import torch
from uformer_pytorch import Uformer

model = Uformer(
    dim = 16,           # initial dimensions after input projection, which increases by 2x each stage
    stages = 4,         # number of stages
    num_blocks = 2,     # number of transformer blocks per stage
    window_size = 16,   # set window size (along one side) for which to do the attention within
    dim_head = 64,
    heads = 1,
    ff_mult = 4
)
model.cuda()
x = torch.randn(1, 3, 512, 512).cuda()
pred = model(x) # (1, 3, 256, 256)

In [ ]:
pred.shape

# Segmentation inference

In [ ]:
start = r"C:\Users\phili\Downloads\Telegram Desktop\Cells\train\labels"
to = r"C:\Users\phili\Downloads\Telegram Desktop\Cells\train\rabs"
for im in os.listdir(start):
    ii = cv2.imread(os.path.join(start, im),0)
    print(np.max(ii))
    #ii = ii*255.
    #cv2.imwrite(os.path.join(to, im), ii)

In [ ]:
model = smp.UnetPlusPlus(encoder_name = "timm-regnety_064", classes = 2, in_channels = 3)
model.load_state_dict(torch.load(r"E:\source\repos\VisualDL\timm-regnety_064, UnetPlusPlus.pt"))
model.eval()

In [ ]:
model = model.cuda()

In [ ]:
test = r"E:\source\repos\HSAData\Evican\train\images"
transform = A.Compose([
    A.Resize(width=512, height=512)
])
for cnt, im in enumerate(os.listdir(test)):
    image = io.imread(os.path.join(test, im))
    image = image/255.
    image = transform(image = image)["image"]
    s = torch.unsqueeze(torch.tensor(image, dtype = torch.float).permute(2, 0, 1), 0).cuda()
    
    preds = model(s)
    preds = torch.argmax(preds, 1)
    preds = preds.detach().cpu().numpy()
    io.imsave(f"{cnt}.png", preds[0])
    
    

# Transformer

In [ ]:
test = r"C:\Users\phili\Downloads\Telegram Desktop\Cells\train\labels"
ab = r"C:\Users\phili\Downloads\Telegram Desktop\Cells\train\bs"
for cnt, im in enumerate(os.listdir(test)):
    img = cv2.imread(os.path.join(test, im)) * 255.
    kernel = np.ones((3, 3), 'uint8')
    dilate_img = cv2.dilate(img, kernel, iterations=1)
    img1_bg = dilate_img - img
    img1 = img1_bg[:,:,0]
    clipped = np.clip(img1, 1, 6) # weight edges by factor (e.g. 6)
    print(np.min(clipped))
    cv2.imwrite(os.path.join(ab, im),clipped)

In [ ]:
from segmentation_models_pytorch.losses import DiceLoss

In [ ]:
DiceLoss(reduce = "none")

# Classification inference

In [ ]:
import timm
from skimage import io
import torch
import os
import numpy as np

In [ ]:
first = timm.create_model("regnety_004", pretrained=True, num_classes = 5).cuda()
#second = timm.create_model("resnext50d_32x4d", pretrained=True, num_classes = 5).cuda()
first.load_state_dict(torch.load(r"E:\source\repos\VisualDL\regnety_004.pt"))
#second.load_state_dict(torch.load(r"E:\source\repos\VisualDL\resnext50d_32x4d.pt"))
first.eval()
#second.eval()

In [ ]:
first = timm.create_model("resnest50d", pretrained=True, num_classes = 5).cuda()
second = timm.create_model("resnext50d_32x4d", pretrained=True, num_classes = 5).cuda()
first.load_state_dict(torch.load(r"E:\source\repos\VisualDL\resnest50d.pt"))
second.load_state_dict(torch.load(r"E:\source\repos\VisualDL\resnext50d_32x4d.pt"))
first.eval()
second.eval()

In [ ]:
path = ""
names = ["NA", "TRG0", "TRG1", "TRG2", "TRG3"]
values = dict()
for name in names:
    values[name] = []
    base = r"C:\Users\phili\Downloads\Telegram Desktop\dataset_valid/" + name
    for im in os.listdir(base):
        image = io.imread(os.path.join(base, im)).astype(np.float32)
        image = image/255.
        s = torch.unsqueeze(torch.tensor(image, dtype = torch.float).permute(2, 0, 1), 0).cuda()
        preds =  first(s)# +  second(s)
        preds = torch.argmax(preds, 1)
        preds = preds.detach().cpu().numpy()
        values[name].append(preds[0])
    

In [ ]:
values

In [ ]:
path = r"C:\Users\phili\Downloads\Telegram Desktop\dataset_valid\TRG2\E93 L X20_0_1183_3925.png"
base = r"C:\Users\phili\Downloads\Telegram Desktop\dataset_valid\TRG2"


image = io.imread(path)
image = image/255.
s = torch.unsqueeze(torch.tensor(image, dtype = torch.float).permute(2, 0, 1), 0).cuda()
preds = first(s) + second(s)
preds = torch.argmax(preds, 1)
preds = preds.detach().cpu().numpy()
print(preds)
    